## Постановка задачи
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Сформируйте параллельный ансамбль из CatBoost, градиентного бустинга, XGBoost и LightGBM. Используйте лучшие гиперпараметры, подобранные ранее, или найдите их через перекрестную проверку. Итоговое решение рассчитайте на основании самого точного предсказания класса у определенной модели ансамбля: выберите для каждого класса модель, которая предсказывает его лучше всего.

Проведите расчеты и выгрузите результат в виде submission.csv

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

### Подключение библиотек

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import StackingClassifier

### Загрузка данных

In [15]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [16]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)
data["Response"] = data["Response"] - 1

### Набор столбцов для расчета

In [17]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [18]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [19]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [20]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [21]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3         4         5         6  \
6214  -0.748535  0.269287 -1.299805 -1.043945  0.611816 -0.169434 -1.159180   
22182  1.601562  1.249023  0.516602  1.050781 -1.634766 -0.169434  0.862305   
55504 -0.748535  0.024353 -0.997070 -0.916992  0.611816 -0.169434 -1.159180   
14822  0.073853 -0.220581  0.365479  0.275879  0.611816 -0.169434  0.862305   
54628 -1.030273 -0.710449 -1.981445 -0.886230 -1.634766 -0.169434  0.862305   

              7         8         9  ...       109      110       111  \
6214   1.100586 -1.156250  1.130859  ... -0.083679  0.44165 -0.149292   
22182 -1.013672  0.864746 -0.928711  ... -0.083679  0.44165 -0.149292   
55504  1.100586 -1.156250  1.130859  ... -0.083679  0.44165 -0.149292   
14822  0.043671  0.861816 -0.928711  ... -0.083679  0.44165 -0.149292   
54628 -1.013672  0.861816 -0.928711  ... -0.083679  0.44165 -0.149292   

            112       113       114       115       116      117  Response  
6214  -0.

### Построение базовых моделей

In [22]:
x_train = pd.DataFrame(data_train, columns=columns_transformed)
y_train = data_train["Response"]

In [25]:
estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('xgb',XGBClassifier(max_depth=17, max_features=27, n_estimators=76, min_samples_leaf=20)),
    ('lgb', lgb.LGBMClassifier(random_state=17, max_depth=18, min_child_samples=17, num_leaves=35, n_estimators=1000, objective="multiclass", num_class=8))]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), n_jobs=-1)

In [26]:
clf.fit(X=x_train, y=y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=42)),
                               ('xgb',
                                XGBClassifier(max_depth=17, max_features=27,
                                              min_samples_leaf=20,
                                              n_estimators=76)),
                               ('lgb',
                                LGBMClassifier(max_depth=18,
                                               min_child_samples=17,
                                               n_estimators=1000, num_class=8,
                                               num_leaves=35,
                                               objective='multiclass',
                                               random_state=17))],
                   final_estimator=LogisticRegression(), n_jobs=-1)

### Оценка ансамбля
Рассчитаем оценку взвешенного предсказания 4 моделей

Кластеризация дает 0.192, kNN(100) - 0.382, лог. регрессия - 0.512/0.496, SVM - 0.95, реш. дерево - 0.3, случайный лес - 0.487, XGBoost - 0.536, градиентный бустинг - 0.56, LightGBM - 0.569, CatBoost - 0.542

In [27]:
x_test = pd.DataFrame(data_test, columns=columns_transformed)
y_test = data_test["Response"]

In [28]:
y_predict = clf.predict(X=x_test)

In [29]:
print(cohen_kappa_score(y_test, y_predict))

0.46714551749702116


### Матрица неточностей

In [30]:
print ("Ансамбль классификации\n",
    confusion_matrix(y_predict, y_test))

Ансамбль классификации
 [[ 352  187   18   15   44  125   60   22]
 [ 146  352    7    1  108  101   35   21]
 [  27   17   73    7    1    0    0    0]
 [  30   32   51  194    0    3    0    2]
 [  96  144   15    0  573   79   12    8]
 [ 263  263   12   15  192 1240  328  137]
 [ 112   94    5    8   63  274  663  180]
 [ 248  236    7   26   87  438  559 3469]]
